In [78]:
import boto3

In [79]:
import dotenv
dotenv.load_dotenv()

True

In [80]:
import os

In [81]:
bt3s = boto3.Session(region_name='us-west-2',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
)
ddb = bt3s.client('dynamodb')

In [82]:
a = ddb.scan(TableName='trip-stream')

In [83]:
len(a)

4

In [84]:
len(a['Items'])

1028

In [96]:
import json

In [105]:
json.loads(a['Items'][0]['stop_time_update_dict']['S'])

{'stop_id': ['LAFY',
  'ORIN',
  'ROCK',
  'MCAR',
  '19TH',
  '12TH',
  'WOAK',
  'EMBR',
  'MONT',
  'POWL',
  'CIVC',
  '16TH',
  '24TH',
  'GLEN',
  'BALB',
  'DALY',
  'COLM',
  'SSAN',
  'SBRN'],
 'delay': [52, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'time_expected': [1701974906,
  1701975173,
  1701975484,
  1701975664,
  1701975868,
  1701975994,
  1701976282,
  1701976702,
  1701976804,
  1701976906,
  1701977008,
  1701977152,
  1701977272,
  1701977440,
  1701977578,
  1701977806,
  1701978070,
  1701978250,
  1701978442]}

In [8]:
values = {
    'timestamp_id': [],
    'time_expected': [],
    'trip_id': [],
    'stop_id': [],
    'delay': []
}
for item in a['Items']:
    delay = int(item['delay']['S'])
    stop_id = item['stop_id']['S']
    trip_id = item['trip_id']['S']
    timestamp_id = int(item['timestamp_id']['S'])
    time_expected = int(item['time_expected']['S'])

    values['delay'].append(delay)
    values['time_expected'].append(time_expected)
    values['trip_id'].append(trip_id)
    values['stop_id'].append(stop_id)
    values['timestamp_id'].append(timestamp_id)


In [9]:
import pandas as pd

In [41]:
df = pd.DataFrame(values).sort_values(by='timestamp_id', ascending=False)

In [11]:
import time

In [12]:
time.ctime(1701964054)

'Thu Dec  7 07:47:34 2023'

In [13]:
df_trips = pd.read_csv('../google_transit_20230911-20240101_v3/trips.txt')

In [14]:
df_stops = pd.read_csv('../google_transit_20230911-20240101_v3/stops.txt')
df_stop_times = pd.read_csv('../google_transit_20230911-20240101_v3/stop_times.txt')
df_routes = pd.read_csv('../google_transit_20230911-20240101_v3/routes.txt')

In [16]:
df_trips_rel = df_trips[['route_id', 'trip_id', 'direction_id']].copy()

In [17]:
df_stop_times_rel = df_stop_times[['trip_id', 'arrival_time', 'stop_id']].copy()

In [18]:
df_routes_rel = df_routes[['route_id', 'route_short_name', 'route_long_name']].copy()

In [19]:
df_stops_rel = df_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].copy()

In [20]:
df_consolidated = df_trips_rel.merge(df_stop_times_rel, how='left', on='trip_id')
df_consolidated = df_consolidated.merge(df_routes_rel, how='left', on='route_id')
df_consolidated = df_consolidated.merge(df_stops_rel, how='left', on='stop_id')

In [21]:
df_consolidated['direction_name'] = df_consolidated['direction_id'].apply(lambda x: 'South' if x ==1 else 'North')

In [22]:
df_consolidated

,route_id,trip_id,direction_id,arrival_time,stop_id,route_short_name,route_long_name,stop_name,stop_lat,stop_lon,direction_name
0,5,1462169,1,04:36:00,SHAY,Green-S,Berryessa/North San Jose to Daly City,South Hayward,37.634363,-122.057158,South
1,5,1462169,1,04:40:00,HAYW,Green-S,Berryessa/North San Jose to Daly City,Hayward,37.669731,-122.086941,South
2,5,1462169,1,04:44:00,BAYF,Green-S,Berryessa/North San Jose to Daly City,Bay Fair,37.696938,-122.126431,South
3,5,1462169,1,04:48:00,SANL,Green-S,Berryessa/North San Jose to Daly City,San Leandro,37.721806,-122.160713,South
4,5,1462169,1,04:52:00,COLS,Green-S,Berryessa/North San Jose to Daly City,Coliseum,37.753604,-122.196698,South
...,...,...,...,...,...,...,...,...,...,...,...
78469,20,1441578,1,24:43:00,OAKL,Beige-S,Coliseum to Oakland Airport,Oakland International Airport Station,37.713256,-122.212237,South
78470,20,1441581,1,24:53:00,COLS,Beige-S,Coliseum to Oakland Airport,Coliseum,37.753604,-122.196698,South
78471,20,1441581,1,25:01:00,OAKL,Beige-S,Coliseum to Oakland Airport,Oakland International Airport Station,37.713256,-122.212237,South
78472,20,1441582,1,25:11:00,COLS,Beige-S,Coliseum to Oakland Airport,Coliseum,37.753604,-122.196698,South


In [35]:
from keplergl import KeplerGl
map = KeplerGl()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [36]:
df_temp = df_stops_rel[~df_stops_rel['stop_name'].str.contains('Enter/Exit')].copy()

In [37]:
map.add_data(data=df_temp, name='stops')

In [38]:
map

KeplerGl(data={'stops': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [107]:
config_to_use = map.config

In [110]:
import pickle

In [112]:
with open('../consumer-service/config_to_use_kepler', 'wb') as f:
    pickle.dump(config_to_use, f)

In [39]:
df_consolidated_rel = df_consolidated[df_consolidated['stop_id'] == 'LAKE']

In [40]:
df_consolidated_rel

,route_id,trip_id,direction_id,arrival_time,stop_id,route_short_name,route_long_name,stop_name,stop_lat,stop_lon,direction_name
6,5,1462169,1,05:00:00,LAKE,Green-S,Berryessa/North San Jose to Daly City,Lake Merritt,37.797357,-122.265232,South
33,5,1462163,1,05:40:00,LAKE,Green-S,Berryessa/North San Jose to Daly City,Lake Merritt,37.797357,-122.265232,South
54,6,1462220,0,05:26:00,LAKE,Green-N,Daly City to Berryessa/North San Jose,Lake Merritt,37.797357,-122.265232,North
72,5,1462168,1,05:20:00,LAKE,Green-S,Berryessa/North San Jose to Daly City,Lake Merritt,37.797357,-122.265232,South
99,5,1462121,1,06:00:00,LAKE,Green-S,Berryessa/North San Jose to Daly City,Lake Merritt,37.797357,-122.265232,South
...,...,...,...,...,...,...,...,...,...,...,...
77756,12,1481459,0,24:05:00,LAKE,Blue-N,Daly City to Dublin/Pleasanton,Lake Merritt,37.797357,-122.265232,North
77771,11,1481401,1,24:16:00,LAKE,Blue-S,Dublin/Pleasanton to Daly City,Lake Merritt,37.797357,-122.265232,South
77792,12,1481460,0,24:25:00,LAKE,Blue-N,Daly City to Dublin/Pleasanton,Lake Merritt,37.797357,-122.265232,North
77807,11,1481402,1,24:36:00,LAKE,Blue-S,Dublin/Pleasanton to Daly City,Lake Merritt,37.797357,-122.265232,South


In [46]:
max(df['timestamp_id'].values)

1701964054

In [75]:
def convert_to_pst(timeepoch):
    utc_time = time.gmtime(timeepoch)
    return f"{utc_time.tm_hour - 8:02}:{utc_time.tm_min:02}"

In [76]:
a = convert_to_pst(1701964054)

In [77]:
a

'07:47'

In [56]:
a.tm_hour - 8, a.tm_min

(7, 47)

In [45]:
df[df['stop_id']=="LAKE"].sort_values(by='timestamp_id', ascending=False).head

,timestamp_id,time_expected,trip_id,stop_id,delay
1355,1701964054,1701964145,1461686,LAKE,42
1145,1701964054,1701964112,1462174,LAKE,69
3872,1701964024,1701964145,1461686,LAKE,42
5324,1701964024,1701964112,1462174,LAKE,69
1542,1701963993,1701964145,1461686,LAKE,42
...,...,...,...,...,...
1470,1701961740,1701961706,1461684,LAKE,4
5043,1701961709,1701961754,1462172,LAKE,110
5475,1701961709,1701961706,1461684,LAKE,4
4973,1701961679,1701961706,1461684,LAKE,4
